# Day 9: Advanced SQL for Energy Data

1. Overview  
2. Copy & Open Database  
3. Common Table Expressions (CTEs)  
4. Window Functions  
5. Date Handling  

---

## 1. Overview

Today you’ll learn to:

- Break queries into logical building blocks with CTEs

- Use window functions (ROW_NUMBER(), RANK(), cumulative sums)

- Turn year+month columns into proper date values in SQL

- Apply these to your generation table from CEA data

---

## 2. Copy & Open Database

```bash
    cp ../day8/cea_gen.db .
    sqlite3 cea_gen.db.
```
---

## 3. Common Table Expressions (CTEs)

```bash
-- Define a CTE for monthly totals
WITH monthly_totals AS (
  SELECT region,
         month,
         SUM(generation) AS total_mu
  FROM generation
  GROUP BY region, month
)
-- Query the CTE
SELECT *
FROM monthly_totals
WHERE total_mu > 800;
```

---

## 4. Window Functions

a) Ranking within partitions
```bash
SELECT
  region,
  month,
  fuel_type,
  generation,
  ROW_NUMBER() OVER (
    PARTITION BY region, month
    ORDER BY generation DESC
  ) AS rank_within_month
FROM generation;
```
b) Cumulative sums
```bash
SELECT
  region,
  month,
  SUM(generation) OVER (
    PARTITION BY region
    ORDER BY month
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
  ) AS cumulative_mu
FROM generation;
```

---

## 5. Date Handling

- Combine year and month into a real SQL date:
```bash
SELECT
  region,
  fuel_type,
  DATE(year || '-' || printf('%02d', month) || '-01') AS period_start,
  SUM(generation) AS total_mu
FROM generation
GROUP BY region, fuel_type, period_start
ORDER BY period_start;
```